In [474]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry_convert as pc

In [34]:
!pip install pycountry_convert

You should consider upgrading via the '/Users/SidharrthNagappan/.pyenv/versions/3.8.2/bin/python3.8 -m pip install --upgrade pip' command.


In [475]:
# see all columns for data
pd.set_option('display.max_columns', 85)

# see all rows for schema
pd.set_option('display.max_rows', 85)

# Data collection and reading

In [476]:
# read raw data
crops_raw = pd.read_csv('cropMainData/ProductionCropsEAllDataNOFLAG.csv', encoding = "ISO-8859-1")
producer_prices = pd.read_csv('prices/Prices_E_All_Data_NOFLAG.csv', encoding = "ISO-8859-1")
caffeine_prices_raw = pd.read_csv('prices/caffeine-crop-prices.csv')
rainfall = pd.read_csv('climate/countries-rainfall-climate.csv')
temperatures = pd.read_csv('climate/countries-temperature-climate.csv')

# include grouping data
country_groups = pd.read_csv('datagroups/country_groups.csv')
item_groups = pd.read_csv('datagroups/item_groups.csv')

# Preparing data

### Main crop data preparation

In [477]:
# extract areas, items and elements and their respective codes into separate dfs
areas = crops_raw.set_index('Area Code')[['Area']].drop_duplicates()
items = crops_raw.set_index('Item Code')[['Item']].drop_duplicates()
elements = crops_raw.set_index('Element Code')[['Element']].drop_duplicates()

# dropping code and unit columns
crops_raw.drop(['Area Code', 'Item Code', 'Element Code', 'Unit'], axis=1, inplace=True)

# remove preceeding 'Y' from the year columns
crops_raw.columns = [name[1:] if name[0]=='Y' else name for name in crops_raw.columns]

In [478]:
# basic cleaning, fill null values with 0 first
crops_clean = crops_raw.fillna(0)

crops_clean # check

,Area,Item,Element,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,"Almonds, with shell",Area harvested,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5900.0,6000.0,6000.0,6000.0,5800.0,5800.0,5800.0,5700.0,5700.0,5600.0,5500.0,5500.0,5400.0,5400.0,6037.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,7000.0,9000.0,5500.0,5700.0,12000.0,11768.0,12000.0,12000.0,12000.0,11029.0,11210.0,13469.0,13490.0,14114.0,13703.0,14676.0,19481.0,19793.0,20053.0
1,Afghanistan,"Almonds, with shell",Yield,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16610.0,15000.0,20000.0,17500.0,17069.0,13793.0,18966.0,17018.0,18421.0,16071.0,18182.0,16364.0,16667.0,16296.0,15736.0,16364.0,18000.0,16364.0,16364.0,16364.0,16364.0,16364.0,16364.0,20000.0,17143.0,16667.0,21407.0,24561.0,12250.0,13281.0,16667.0,26234.0,35000.0,39154.0,49955.0,45000.0,45960.0,29910.0,19996.0,16521.0,16859.0,13788.0,17161.0
2,Afghanistan,"Almonds, with shell",Production,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9800.0,9000.0,12000.0,10500.0,9900.0,8000.0,11000.0,9700.0,10500.0,9000.0,10000.0,9000.0,9000.0,8800.0,9500.0,9000.0,9900.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,11000.0,12000.0,15000.0,11774.0,14000.0,14700.0,15630.0,20000.0,31481.0,42000.0,43183.0,56000.0,60611.0,62000.0,42215.0,27400.0,24246.0,32843.0,27291.0,34413.0
3,Afghanistan,"Anise, badian, fennel, coriander",Area harvested,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0,700.0,300.0,1100.0,1300.0,1882.0,700.0,2270.0,2871.0,3000.0,7000.0,4000.0,12000.0,11311.0,4000.0,1600.0,3300.0,6800.0,15000.0,16000.0,17432.0,28000.0,15000.0,17748.0,17000.0,19500.0,18500.0,18500.0,30000.0,25000.0,25638.0,27582.0,25785.0
4,Afghanistan,"Anise, badian, fennel, coriander",Yield,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7143.0,7143.0,6667.0,7273.0,7692.0,7072.0,7143.0,6854.0,6761.0,6667.0,6000.0,6250.0,5917.0,6189.0,6250.0,6250.0,6061.0,6029.0,6000.0,6250.0,6222.0,6071.0,6000.0,6203.0,6000.0,6414.0,6757.0,6757.0,7167.0,7200.0,7037.0,6954.0,7036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50246,Net Food Importing Developing Countries,"Treenuts, Total",Yield,6118.0,6318.0,6618.0,6492.0,6417.0,6187.0,5978.0,5595.0,5954.0,5381.0,5603.0,5788.0,6052.0,6049.0,5916.0,5885.0,5906.0,6140.0,6115.0,6000.0,6144.0,6208.0,4931.0,4864.0,5623.0,5491.0,5456.0,5351.0,5486.0,5680.0,5595.0,5546.0,5283.0,5164.0,5118.0,5487.0,5316.0,5829.0,5705.0,5566.0,5881.0,4765.0,5008.0,4903.0,5300.0,4935.0,4812.0,5321.0,4728.0,5415.0,5429.0,5417.0,5198.0,5177.0,5234.0,5514.0,5251.0,5461.0
50247,Net Food Importing Developing Countries,"Treenuts, Total",Production,255724.0,295318.0,315667.0,349663.0,341957.0,336409.0,318793.0,412051.0,371776.0,434382.0,474040.0,476912.0,555059.0,521422.0,495377.0,406926.0,407359.0,370156.0,348867.0,354970.0,407357.0,381473.0,309000.0,344568.0,354345.0,350584.0,353961.0,366219.0,412978.0,392186.0,426378.0,468381.0,441800.0,442583.0,514115.0,579467.0,600857.0,677838.0,789032.0,790481.0,852956.0,774584.0,798242.0,859369.0,994559.0,1051460.0,1145260.0,1316822.0,1338811.0,1370375.0,1619567.0,1662130.0,1772637.0,1835843.0,2090427.0,1953370.0,2076194.0,2219089.0
50248,Net Food Importing Developing Countries,Vegetables Primary,Area harvested,1877800.0,1925219.0,1992435.0,2054949.0,2039738.0,2138112.0,2172385.0,2227854.0,2218532.0,2267116.0,2299393.0,2319485.0,2360046.0,2443375.0,2536820.0,2634309.0,2610304.0,2637054

### Handling country groups (continents)

In [147]:
# looking at crop data 'Area' field, shows that every item before 'World' is a country
print('\nAn array of all the area names in crop data:')
print(crops_clean['Area'].unique())

# get the index of 'World' and hence the total number of countries
areas_ser = pd.Series(crops_clean['Area'].unique()) # taking unique areas
wi = areas_ser[areas_ser=='World'].index[0] 
print(f'Total number of countries in crop data: {wi}\n') # show number of countries

# extract countries
allCountries = areas_ser[areas_ser.index[0:wi]]
del areas_ser
allCountries # all countries in crops data


An array of all the area names in crop data:
['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belgium-Luxembourg' 'Belize' 'Benin' 'Bermuda' 'Bhutan'
 'Bolivia (Plurinational State of)' 'Bosnia and Herzegovina' 'Botswana'
 'Brazil' 'British Virgin Islands' 'Brunei Darussalam' 'Bulgaria'
 'Burkina Faso' 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada'
 'Cayman Islands' 'Central African Republic' 'Chad' 'Chile' 'China'
 'China, Hong Kong SAR' 'China, Macao SAR' 'China, mainland'
 'China, Taiwan Province of' 'Colombia' 'Comoros' 'Congo' 'Cook Islands'
 'Costa Rica' "Côte d'Ivoire" 'Croatia' 'Cuba' 'Cyprus' 'Czechia'
 'Czechoslovakia' "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Es

0         Afghanistan
1             Albania
2             Algeria
3      American Samoa
4              Angola
            ...      
219    Western Sahara
220             Yemen
221      Yugoslav SFR
222            Zambia
223          Zimbabwe
Length: 224, dtype: object

In [148]:
# view the country_groups data
# check for country groups
print(country_groups['Country Group'].value_counts())

World                                      225
Net Food Importing Developing Countries     79
Africa                                      57
Low Income Food Deficit Countries           53
Asia                                        52
Europe                                      50
Small Island Developing States              50
Least Developed Countries                   49
Americas                                    46
Land Locked Developing Countries            32
European Union (28)                         29
European Union (27)                         28
Caribbean                                   20
Oceania                                     20
Eastern Africa                              20
Western Asia                                18
Western Africa                              16
Northern Europe                             14
Southern Europe                             14
South America                               14
Eastern Europe                              12
South-eastern

In [149]:
# list of continents (UN geoscheme based on M49 coding)
continents_list = ['Africa', 'Asia', 'Europe', 'Americas', 'Oceania']

# create a country continent table.. take only the country-continent rows
ccTable = pd.DataFrame(country_groups[country_groups['Country Group'].isin(continents_list)])
ccTable.drop(columns=['Country Group Code', 'Country Code', 'M49 Code', 'ISO2 Code', 'ISO3 Code'], inplace=True)
ccTable.reset_index(drop=True, inplace=True)
ccTable.rename(columns = {'Country Group': 'Continent'}, inplace = True)

In [155]:
# countries in allallCountries but not in ccTable
print(allCountries[~allCountries.isin(ccTable['Country'])])

3                                    American Samoa
84                                             Guam
148                 Pacific Islands Trust Territory
166    Saint Helena, Ascension and Tristan da Cunha
211                    United States Virgin Islands
218                       Wallis and Futuna Islands
219                                  Western Sahara
dtype: object


### Rainfall and temperature data cleaning

In [479]:
# removing leading and trailing whitespaces in columns
rainfall.columns = [col.strip() for col in rainfall.columns]
temperatures.columns = [col.strip() for col in temperatures.columns]

In [480]:
# list of countries in rainfall with naming discrepencies with allCountries
rainfall.loc[~rainfall['Country'].isin(allCountries), 'Country'].unique()

array([' Afghanistan', ' Albania', ' Algeria', ' Andorra', ' Angola',
       ' Antigua and Barbuda', ' Argentina', ' Armenia', ' Australia',
       ' Austria', ' Azerbaijan', ' Bahamas', ' Bahrain', ' Bangladesh',
       ' Barbados', ' Belarus', ' Belgium', ' Belize', ' Benin',
       ' Bhutan', ' Bolivia', ' Bosnia and Herzegovina', ' Botswana',
       ' Brazil', ' Brunei', ' Bulgaria', ' Burkina Faso', ' Burundi',
       ' Cambodia', ' Cameroon', ' Canada', ' Cape Verde',
       ' Central African Republic', ' Chad', ' Chile', ' China',
       ' Colombia', ' Comoros', ' Congo (Democratic Republic of the)',
       ' Congo (Republic of the)', ' Costa Rica', " Cote d'Ivoire",
       ' Croatia', ' Cuba', ' Cyprus', ' Czech Republic', ' Denmark',
       ' Djibouti', ' Dominica', ' Dominican Republic', ' Ecuador',
       ' Egypt', ' El Salvador', ' Equatorial Guinea', ' Eritrea',
       ' Estonia', ' Ethiopia', ' Faroe Islands',
       ' Federated States of Micronesia', ' Fiji', ' Finland',

In [481]:
# list of countries in temperature with naming discrepencies with allCountries
temperatures.loc[~temperatures['Country'].isin(allCountries), 'Country'].unique()

array([' Afghanistan', ' Albania', ' Algeria', ' Andorra', ' Angola',
       ' Antigua and Barbuda', ' Argentina', ' Armenia', ' Australia',
       ' Austria', ' Azerbaijan', ' Bahamas', ' Bahrain', ' Bangladesh',
       ' Barbados', ' Belarus', ' Belgium', ' Belize', ' Benin',
       ' Bhutan', ' Bolivia', ' Bosnia and Herzegovina', ' Botswana',
       ' Brazil', ' Brunei', ' Bulgaria', ' Burkina Faso', ' Burundi',
       ' Cambodia', ' Cameroon', ' Canada', ' Cape Verde',
       ' Central African Republic', ' Chad', ' Chile', ' China',
       ' Colombia', ' Comoros', ' Congo (Democratic Republic of the)',
       ' Congo (Republic of the)', ' Costa Rica', " Cote d'Ivoire",
       ' Croatia', ' Cuba', ' Cyprus', ' Czech Republic', ' Denmark',
       ' Djibouti', ' Dominica', ' Dominican Republic', ' Ecuador',
       ' Egypt', ' El Salvador', ' Equatorial Guinea', ' Eritrea',
       ' Estonia', ' Ethiopia', ' Faroe Islands',
       ' Federated States of Micronesia', ' Fiji', ' Finland',

In [482]:
# tester cell
print(allCountries[allCountries.str.contains('Monaco')])

Series([], dtype: object)


In [483]:
# dictionary for country name adjustments
d = {'Bolivia':'Bolivia (Plurinational State of)', 
     'Brunei':'Brunei Darussalam',
     'Cape Verde': 'Cabo Verde',
    'Congo (Democratic Republic of the)':'Democratic Republic of the Congo',
    'Congo (Republic of the)': 'Congo',
    "Cote d'Ivoire":"Côte d'Ivoire",
    'Czech Republic':'Czechia',
    'Federated States of Micronesia':'Micronesia (Federated States of)',
    'Iran':'Iran (Islamic Republic of)',
    'Korea':'Republic of Korea',
    'Laos':"Lao People's Democratic Republic",
    'Macedonia':'North Macedonia',
    'Moldova':'Republic of Moldova',
    'Myanmar (Burma)':'Myanmar',
    'Republic of Montenegro':'Serbia and Montenegro',
    'Republic of Serbia':'Serbia',
    'Russia':'Russian Federation', 
    'St. Kitts and Nevis':'Saint Kitts and Nevis', 
    'St. Lucia':'Saint Lucia',
    'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines',
    'Swaziland':'Eswatini', 
    'Syria':'Syrian Arab Republic', 
    'Tanzania':'United Republic of Tanzania',
    'Timor Leste':'Timor-Leste', 
    'United Kingdom':'United Kingdom of Great Britain and Northern Ireland', 
    'United States':'United States of America', 
    'Venezuela':'Venezuela (Bolivarian Republic of)',
    'Vietnam':'Viet Nam'}

In [484]:
# fixing discrepencies
rainfall['Country'] = rainfall['Country'].apply(str.strip)
rainfall['Country'] = rainfall['Country'].replace(d)
temperatures['Country'] = temperatures['Country'].apply(str.strip)
temperatures['Country'] = temperatures['Country'].replace(d)

In [485]:
# checking again, leave these ones ones out (no crop data!)
rainfall.loc[~rainfall['Country'].isin(allCountries), 'Country'].unique()
temperatures.loc[~temperatures['Country'].isin(allCountries), 'Country'].unique() 

array(['Andorra', 'Greenland', 'Monaco', 'Northern Mariana Islands',
       'Palau'], dtype=object)

### Cleaning for Item groups

In [486]:
item_groups['Item Group'].value_counts() # certain groups overlap and not all crops have groups

Crops Primary                160
Fruit Primary                 39
Vegetables Primary            24
Oilcrops, Oil Equivalent      22
Oilcrops                      21
Oilcrops, Cake Equivalent     16
Cereals, Total                15
Pulses, Total                 11
Fibre Crops Primary           10
Treenuts, Total                8
Roots and Tubers, Total        7
Citrus Fruit, Total            5
Sugar Crops Primary            3
Name: Item Group, dtype: int64

In [488]:
# getting the unique groups..

# dropping overlapping groups
unique_groups = item_groups.copy()
unique_groups.drop(unique_groups[unique_groups['Item Group'] == 'Crops Primary'].index, inplace = True)
unique_groups.drop(unique_groups[unique_groups['Item Group'] == 'Oilcrops'].index, inplace = True)
unique_groups.drop(unique_groups[unique_groups['Item Group'] == 'Oilcrops, Cake Equivalent'].index, inplace = True)
unique_groups.drop(unique_groups[unique_groups['Item Group'] == 'Citrus Fruit, Total'].index, inplace = True)

unique_groups
unique_groups['Item Group'].unique()

array(['Cereals, Total', 'Fibre Crops Primary', 'Fruit Primary',
       'Oilcrops, Oil Equivalent', 'Pulses, Total',
       'Roots and Tubers, Total', 'Sugar Crops Primary',
       'Treenuts, Total', 'Vegetables Primary'], dtype=object)

In [489]:
# a funtion to return the crop group given the crop
def getItemType(item):
    g = ug.loc[ug['Item'] == item, 'Item Group']
    if g.empty:
        return np.nan
    else:
        return g.iloc[0]

# create a new df from crops, unique groups only
all_items = crops_clean['Item'].unique()
all_item_groups = [ getItemType(x) for x in all_items ]
crop_groups = pd.DataFrame({'crop': all_items, 'crop_type': all_item_groups})

# better group names
group_names = {
    'Cereals, Total' : 'Cereals',
    'Fibre Crops Primary': 'Fibre Crops', 
    'Fruit Primary' : 'Fruit',
    'Oilcrops, Oil Equivalent': 'Oilcrops', 
    'Pulses, Total': 'Pulses',
    'Roots and Tubers, Total': 'Roots and Tubers',
    'Sugar Crops Primary' : 'Sugar Crops',
    'Treenuts, Total': 'Treenuts', 
    'Vegetables Primary': 'Vegetables'
}

crop_groups['crop_type'] = crop_groups['crop_type'].map(group_names)

crop_groups[crop_groups['crop_type'].isna()] # still some missing groups

,crop,crop_type
1,"Anise, badian, fennel, coriander",NaN
26,"Rice, paddy (rice milled equivalent)",NaN
29,Spices nes,NaN
37,"Cereals, Total",NaN
38,"Citrus Fruit, Total",NaN
39,Fibre Crops Primary,NaN
40,Fruit Primary,NaN
41,Oilcrops,NaN
42,"Oilcrops, Cake Equivalent",NaN
43,"Oilcrops, Oil Equivalent",NaN


In [490]:
crop_groups.drop([x for x in range (37,49)],inplace=True) # drop rosw with old groups from all_items
crop_groups.fillna('Others', inplace=True) # create a new group for crops without one
crop_groups.reset_index(drop=True, inplace=True)
crop_groups

,crop,crop_type
0,"Almonds, with shell",Treenuts
1,"Anise, badian, fennel, coriander",Others
2,Apples,Fruit
3,Apricots,Fruit
4,Barley,Cereals
...,...,...
158,Tallowtree seed,Oilcrops
159,Agave fibres nes,Fibre Crops
160,Manila fibre (abaca),Fibre Crops
161,Kapok fruit,Fibre Crops


In [ ]:
# side note: maybe can futher divide 'Others' into spices and caffeine crops

### Producer (by country) crop price dataset preparation

In [491]:
producer_prices

,Area Code,Area,Item Code,Item,Element Code,Element,Months Code,Months,Unit,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,221,"Almonds, with shell",5530,Producer Price (LCU/tonne),7021,Annual value,LCU,NaN,NaN,46000.00,50000.00,62000.00,50000.00,41000.00,42000.00,52000.00,67000.00,83000.00,75000.00,89000.00,91000.00,90200.00,NaN,NaN,213810.00,186000.00,215700.00,230900.00,240500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Afghanistan,221,"Almonds, with shell",5531,Producer Price (SLC/tonne),7021,Annual value,SLC,NaN,NaN,46000.00,50000.00,62000.00,50000.00,41000.00,42000.00,52000.00,67000.00,83000.00,75000.00,89000.00,91000.00,90200.00,NaN,NaN,213810.00,186000.00,215700.00,230900.00,240500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Afghanistan,221,"Almonds, with shell",5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,21.57,21.57,21.57,23.44,29.07,23.44,19.22,19.69,24.38,31.42,38.92,35.17,41.73,42.67,42.29,61.61,80.93,100.25,87.21,101.14,108.27,112.77,108.92,105.42,101.22,93.36,89.57,85.02,NaN
3,2,Afghanistan,711,"Anise, badian, fennel, coriander",5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,10.62,10.62,10.62,10.62,19.72,26.69,33.37,40.65,50.36,51.57,49.75,50.36,50.97,54.68,59.40,59.72,70.09,67.98,72.16,100.43,111.16,112.77,108.92,105.42,101.22,93.36,89.57,85.02,NaN
4,2,Afghanistan,515,Apples,5530,Producer Price (LCU/tonne),7021,Annual value,LCU,NaN,NaN,6100.00,6200.00,4500.00,16000.00,18500.00,12800.00,24800.00,12500.00,12000.00,15000.00,14000.00,13200.00,11000.00,NaN,NaN,21740.00,25000.00,25070.00,24200.00,22600.00,NaN,NaN,NaN,31400.00,35700.00,22331.40,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93929,181,Zimbabwe,1732,"Oilcrops, Oil Equivalent",5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,0.41,0.57,0.82,0.97,1.58,1.60,2.54,2.19,4.84,7.25,11.13,28.97,348.85,1347.85,6966.32,98464.50,333948.69,NaN,NaN,59.03,85.54,100.97,100.97,105.34,92.16,102.98,113.87,NaN,NaN
93930,181,Zimbabwe,1726,"Pulses, Total",5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,0.14,0.14,0.78,0.83,1.30,1.97,1.91,1.46,2.57,9.07,15.72,74.08,384.77,3115.07,7889.50,109848.13,390122.01,NaN,NaN,67.17,71.64,88.76,89.55,114.20,82.67,103.13,103.32,NaN,NaN
93931,181,Zimbabwe,1720,"Roots and Tubers, Total",5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,0.53,0.73,0.59,0.59,0.57,0.73,1.16,1.75,2.54,47.78,93.90,15.13,230.36,519.13,2053.94,64668.17,189710.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.76,101.24,99.57,NaN,NaN
93932,181,Zimbabwe,1735,Vegetables Primary,5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,0.37,0.48,0.49,0.67,0.72,0.96,1.41,1.45,2.11,2.66,7.05,14.40,128.46,599.89,2248.40,12107.33,36376.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.98,137.02,61.07,NaN,NaN


In [492]:
# checking 'Months' column
producer_prices['Months'].unique()

array(['Annual value', 'January', 'February', 'March', 'April', 'May',
       'June', 'July', 'August', 'September', 'October', 'November',
       'December'], dtype=object)

In [493]:
# checking 'Units' column
producer_prices['Element'].unique()

array(['Producer Price (LCU/tonne)', 'Producer Price (SLC/tonne)',
       'Producer Price Index (2014-2016 = 100)',
       'Producer Price (USD/tonne)'], dtype=object)

In [494]:
# filtering
filt1 = producer_prices['Months']=='Annual value' # filter for only annual values
filt2 = producer_prices['Element']=='Producer Price (USD/tonne)' # filter for usd values
prices_annual = pd.DataFrame(producer_prices.loc[filt1 & filt2])

# dropping unnecessary columns
prices_annual.drop(['Area Code', 'Item Code', 'Element Code', 'Element', 'Months Code', 'Months', 'Unit'], axis=1, inplace=True)

# remove preceeding 'Y' from the year columns
prices_annual.columns = [name[1:] if name[0]=='Y' else name for name in prices_annual.columns]

In [495]:
# basic cleaning, fill null values with 0 first
prices_clean = prices_annual.fillna(0)

prices_clean # check

,Area,Item,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
131,Albania,Apples,0.0,0.0,461.0,0.0,323.6,325.4,253.1,248.9,270.2,257.5,257.9,274.7,317.6,375.6,390.5,428.1,486.6,476.8,463.3,356.0,386.5,369.7,492.1,474.8,389.0,381.3,461.6,435.2,346.7
141,Albania,Apricots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,891.2,530.1,486.6,452.9,526.4,452.2,584.8,489.9,823.3,758.4,746.3,732.1,599.1,777.8,909.8
157,Albania,Barley,0.0,0.0,0.0,0.0,0.0,287.1,248.4,325.3,305.0,313.1,299.7,306.8,352.9,401.8,400.5,305.8,337.3,464.9,410.6,307.9,339.0,316.1,482.6,431.4,341.4,319.7,303.6,302.2,306.1
173,Albania,"Beans, dry",0.0,0.0,447.0,0.0,873.8,602.9,725.2,956.0,806.2,528.8,808.4,770.6,593.3,1053.7,1173.5,1181.4,1017.4,1609.2,1642.5,1356.6,1328.1,1562.1,2403.7,2059.6,1738.6,1661.7,1710.4,2078.7,1978.8
189,Albania,"Beans, green",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1315.4,753.7,0.0,767.5,848.8,976.7,1090.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93853,Zimbabwe,Sweet potatoes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325.0,350.0,330.0,0.0,0.0
93871,Zimbabwe,"Tobacco, unmanufactured",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2890.0,2730.1,3670.0,0.0,3160.0,2500.0,2950.0,2960.0,2920.0,0.0
93887,Zimbabwe,Tomatoes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,504.0,567.0,786.0,0.0,0.0
93903,Zimbabwe,"Vegetables, fresh nes",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,671.0,1727.0,607.0,0.0,0.0


### Preparing separate DataFrame for caffeine crops prices

In [496]:
# list of crops that are 3 major sources of caffeine
caffeine_crops = ['Coffee, green', 'Tea', 'Cocoa, beans']

In [497]:
# filtering...
filt1 = caffeine_prices_raw['Item'].isin(caffeine_crops)
filt2 = caffeine_prices_raw['Element'] == 'Producer Price (USD/tonne)'

caffeine_prices = pd.DataFrame(caffeine_prices_raw.loc[filt1 & filt2])
caffeine_prices

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
66,PP,Producer Prices,7,Angola,5532,Producer Price (USD/tonne),656,"Coffee, green",2010,2010,7021,Annual value,USD,511.4,NaN,Official data
67,PP,Producer Prices,7,Angola,5532,Producer Price (USD/tonne),656,"Coffee, green",2011,2011,7021,Annual value,USD,702.6,NaN,Official data
92,PP,Producer Prices,9,Argentina,5532,Producer Price (USD/tonne),667,Tea,1991,1991,7021,Annual value,USD,49.8,NaN,Official data
93,PP,Producer Prices,9,Argentina,5532,Producer Price (USD/tonne),667,Tea,1992,1992,7021,Annual value,USD,50.5,NaN,Official data
94,PP,Producer Prices,9,Argentina,5532,Producer Price (USD/tonne),667,Tea,1993,1993,7021,Annual value,USD,63.1,NaN,Official data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7077,PP,Producer Prices,249,Yemen,5532,Producer Price (USD/tonne),656,"Coffee, green",2013,2013,7021,Annual value,USD,7957.9,NaN,Official data
7078,PP,Producer Prices,249,Yemen,5532,Producer Price (USD/tonne),656,"Coffee, green",2015,2015,7021,Annual value,USD,8520.5,NaN,Official data
7079,PP,Producer Prices,249,Yemen,5532,Producer Price (USD/tonne),656,"Coffee, green",2016,2016,7021,Annual value,USD,9116.2,NaN,Official data
7080,PP,Producer Prices,249,Yemen,5532,Producer Price (USD/tonne),656,"Coffee, green",2017,2017,7021,Annual value,USD,9571.9,NaN,Official data


In [498]:
# dropping redundant columns and renaming one price column
caffeine_prices.drop(['Domain Code', 'Domain', 'Area Code', 'Element Code', 'Year Code', 'Months Code', 'Months', 'Flag', 'Flag Description', 'Element', 'Item Code', 'Unit'], axis=1, inplace=True)
caffeine_prices.rename(columns={'Value':'Price'}, inplace=True)

In [501]:
# replacing missing values
caffeine_prices_clean = caffeine_prices.fillna(0)

caffeine_prices_clean # check

,Area,Item,Year,Price
66,Angola,"Coffee, green",2010,511.4
67,Angola,"Coffee, green",2011,702.6
92,Argentina,Tea,1991,49.8
93,Argentina,Tea,1992,50.5
94,Argentina,Tea,1993,63.1
...,...,...,...,...
7077,Yemen,"Coffee, green",2013,7957.9
7078,Yemen,"Coffee, green",2015,8520.5
7079,Yemen,"Coffee, green",2016,9116.2
7080,Yemen,"Coffee, green",2017,9571.9


In [502]:
#map the country to continent since the data doesn't group regionally
def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return 'N/A'

# Example
caffeine_prices_clean['continent'] = caffeine_prices_clean['Area'].apply(lambda x: country_to_continent(x))
caffeine_prices_clean['continent'].value_counts()

Africa           500
Asia             447
North America    426
South America    309
N/A              181
Oceania           41
Europe             4
Name: continent, dtype: int64

In [504]:
caffeine_prices_clean.continent.isnull().sum()

0

# Saving cleaned data

In [505]:
crops_clean.to_csv('datasets_cleaned/crops_cleaned.csv', index=False, encoding='utf-8')
crop_groups.to_csv('datasets_cleaned/crop_groups.csv', index=False, encoding='utf-8')
prices_clean.to_csv('datasets_cleaned/producer_prices_cleaned.csv', index=False, encoding='utf-8')
caffeine_prices_clean.to_csv('datasets_cleaned/caffeine_prices_clean.csv', index=False, encoding='utf-8')
rainfall.to_csv('datasets_cleaned/rainfall_clean.csv', index=False, encoding='utf-8')
temperatures.to_csv('datasets_cleaned/temperatures_clean.csv', index=False, encoding='utf-8')